In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

sys.path.append("../analysis/scripts")
from plotting_tools import *

In [2]:
df = pd.read_excel("data/UpToDate_Results Sewage PCR_TABLE 27.03.23.xlsx")

In [3]:
df['Date'] = df['Date'].ffill()
df['Date'] = pd.to_datetime(df['Date'])


# select relevant rows / columns
df = df.loc[df["Unnamed: 2"]=="Copies/liter",:]
df_cleaned = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df_cleaned = df_cleaned.loc[:, ['Date', 'Comment', 'Langrieder \nBach',
       'Gräfelfinger Str. \n/Waldwiesenstr.', 'Schenkendorfstr.',
       'Schmidbartlanger ', 'Savitsstr.', 'Gyßling Becken']]

# select relevant dates
df_cleaned = df_cleaned.loc[df_cleaned.Date <= "2020-06-01",:]

In [4]:
df_cleaned = pd.melt(df_cleaned, id_vars=['Date', 'Comment'], var_name='Location', value_name='Value')

In [5]:
df_cleaned.Location = df_cleaned.Location.replace({'Langrieder \nBach': "Am Langwieder Bach",
                                  'Gräfelfinger Str. \n/Waldwiesenstr.': "Gräfelfinger Str.",
                                  "Schmidbartlanger ": "Schmidbartlangerstr.", 
                                  "Savitsstr.": "Savitstr.", 
                                  "Gyßling Becken": "Gyßlinger Becken"})
df_cleaned["Location_Clear"] = df_cleaned["Location"]

df_cleaned["Below_threshold"] = False
df_cleaned.loc[df_cleaned.Value=="<75", "Below_threshold"] = True
df_cleaned.loc[df_cleaned.Value=="<75", "Value"] = 75/2
df_cleaned.Value = df_cleaned.Value.astype(float)
df_cleaned.Date = pd.to_datetime(df_cleaned.Date)

manhole_clear_names_renamed = {x.replace("\n", " "): manhole_clear_names[x] for x in manhole_clear_names.keys()}
df_cleaned["Location"] = df_cleaned["Location"].replace(manhole_clear_names_renamed)

In [6]:
df_cleaned.head()

,Date,Comment,Location,Value,Location_Clear,Below_threshold
0,2020-04-16,NaN,N_Uc,37.500000,Am Langwieder Bach,True
1,2020-04-28,NaN,N_Uc,788.836775,Am Langwieder Bach,False
2,2020-05-05,NaN,N_Uc,37.500000,Am Langwieder Bach,True
3,2020-05-19,NaN,N_Uc,37.500000,Am Langwieder Bach,True
4,2020-05-26,NaN,N_Uc,37.500000,Am Langwieder Bach,True


In [13]:
df_cleaned.to_csv("preprocessed_data/concentration_measurements.csv", index=False)